In [34]:
from skimage import data, io, segmentation, color, util, img_as_ubyte
from skimage import graph
import numpy as np
import cv2 as cv
import os
from scipy.spatial import KDTree

compactness = 5 #Higher values make the shape of the segments more square/cubic. Lower values make the shape of the segments more circular/spherical.
n_segments = 2000 #Desired number of segments. Lower values result in larger segments, and higher values create more, smaller segments. Segments might be fewer than set value, but not more.
thresh = 50 # Higher value results in fewer and larger segments. Lower values result in more and smaller segments.


# parameters for contour detection
contour_color = (255, 0, 0) # color of the contour lines
t_lower = 50 # Lower Threshold
t_upper = 130  # Upper threshold
cont_thickness = 1 #Contour Thickness

images = [{'image_name': 'vermeer', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'unknown 1640', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'turner', 'image_file_format': '.webp', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'leo putz_Halbindianerin mit Früchten', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'cavalcanti_mulata against a green background', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'bazille_family reunion', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'alfred-sisley_autumn-banks-of-the-seine-near-bougival-1873', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'algernon-talmage_the-lake-district-for-holidays-honister-crag-london-midland-and-scottish-railway-poster-artwork', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'armand-guillaumin_echo-rock-1905', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'benjamin-brown_eucalypti-near-arch-beach-california', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'benjamin-brown_grand-canyon', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'benjamin-brown_poppies-and-eucalyptus', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'cezanne_still life with apples_1895', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'kahlo_viva la vida_1954', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'Linda10 (1)', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'Ljus från lampa och omgivning', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'VAZQUEZ_Emmanuel_Honorato_01-scaled', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'warhol_Capa do álbum Velvet Underground & Nico_1967', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'warhol_flowers_1970', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'warhol_mao', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'warhol_self portrait_1972', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'warhol_sunset_1972', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'warhol_vesuvius_1985', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'kaggle dataset1', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'kaggle dataset2', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'kaggle dataset3', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'kaggle dataset4', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},
          {'image_name': 'kaggle dataset5', 'image_file_format': '.jpg', 'compactness': 5, 'n_segments': 2000, 'thresh': 50, 't_lower': 50, 't_upper': 130, 'cont_thickness': 1},   
          ]

predefined_colors = np.array([
    [208, 72, 154], [236, 24, 121], [236, 28, 35], [250, 164, 23], [241, 235, 32], [154, 201, 59], [113, 193, 82], [111, 197, 164], [59, 184, 235], [67, 111, 182], [90, 80, 162], [134, 80, 160],
    [214, 111, 171], [237, 95, 141], [240, 90, 65], [251, 181, 75], [244, 235, 98], [173, 211, 96], [141, 199, 116], [136, 209, 181], [109, 192, 236], [101, 133, 193], [117, 105, 174], [154, 108, 177],
    [223, 144, 189], [243, 135, 166], [243, 134, 99], [254, 197, 119], [248, 239, 139], [191, 222, 135], [170, 213, 150], [171, 219, 196], [149, 207, 242], [134, 153, 205], [143, 134, 190], [171, 137, 190],
    [230, 179, 209], [247, 174, 193], [248, 168, 141], [255, 215, 161], [249, 244, 176], [212, 228, 174], [196, 225, 181], [196, 228, 213], [183, 221, 244], [169, 182, 220], [175, 167, 210], [196, 171, 211],
    [241, 214, 232], [250, 213, 221], [253, 210, 191], [254, 234, 204], [252, 249, 215], [231, 241, 212], [225, 239, 216], [226, 240, 232], [220, 236, 250], [209, 214, 236], [209, 206, 233], [222, 210, 229]
  ])


def closest_predefined_color(mean_color, predefined_colors):
    tree = KDTree(predefined_colors)
    _, idx = tree.query(mean_color)
    return predefined_colors[idx]

def _weight_mean_color(graph, src, dst, n):
    """Callback to handle merging nodes by recomputing mean color.

    The method expects that the mean color of `dst` is already computed.

    Parameters
    ----------
    graph : RAG
        The graph under consideration.
    src, dst : int
        The vertices in `graph` to be merged.
    n : int
        A neighbor of `src` or `dst` or both.

    Returns
    -------
    data : dict
        A dictionary with the `"weight"` attribute set as the absolute
        difference of the mean color between node `dst` and `n`.
    """

    diff = graph.nodes[dst]['mean color'] - graph.nodes[n]['mean color']
    diff = np.linalg.norm(diff)
    return {'weight': diff}


def merge_mean_color(graph, src, dst):
    """Callback called before merging two nodes of a mean color distance graph.

    This method computes the mean color of `dst`.

    Parameters
    ----------
    graph : RAG
        The graph under consideration.
    src, dst : int
        The vertices in `graph` to be merged.
    """
    graph.nodes[dst]['total color'] += graph.nodes[src]['total color']
    graph.nodes[dst]['pixel count'] += graph.nodes[src]['pixel count']
    graph.nodes[dst]['mean color'] = (graph.nodes[dst]['total color'] /
                                      graph.nodes[dst]['pixel count'])


# Define a function to make nearly black parts transparent
def make_black_transparent(image_data, threshold=50):
    # Extract the RGB channels
    r, g, b, a = np.rollaxis(image_data, axis=-1)
    # Create a mask for nearly black pixels
    mask = (r < threshold) & (g < threshold) & (b < threshold)
    # Set the alpha channel to 0 for nearly black pixels
    image_data[mask, 3] = 0
    return image_data


# Convert an RGB color from 0-255 range to 0-1 range
def convert_color_range(color):
    return tuple([c / 255.0 for c in color])


def get_contours(image): 

    cont_color = (0, 0, 255)  # BGR color for contour lines, red in this case

    print("Reading image: " + image)
    im = cv.imread(image)
    assert im is not None, "File could not be read, check with os.path.exists()"
    print("Converting to grayscale")
    imgray = cv.cvtColor(im, cv.COLOR_BGR2GRAY)
    print("Done")
    blur = cv.GaussianBlur(im, (3, 3), 0)

    # Applying the Canny Edge filter
    print("Finding edges")
    edge = cv.Canny(blur, t_lower, t_upper)
    contours, hierarchy = cv.findContours(edge, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    print(f"Drawing contours with thickness {cont_thickness}")
    # Create a new black image of the same size as the original
    black_image = np.zeros_like(im)

    # Drawing the contours on the black image
    cv.drawContours(black_image, contours, -1, cont_color, cont_thickness)

    # Ensure the output directory exists
    output_dir = 'output_images'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Saving the image with contours on black background
    image_name = os.path.splitext(os.path.basename(image))[0]
    full_save_path = os.path.join(output_dir, image_name + ' -- contours.png')
    print(f"Saving image to: {full_save_path}")
    cv.imwrite(full_save_path, black_image)

    # Displaying the image for verification
    # cv.imshow('Contours', black_image)
    # print("Press any key to close the image window.")
    # cv.waitKey(0)  # Waits indefinitely for a key press
    # cv.destroyAllWindows()  # Closes all the OpenCV windows

for image in images:
    image_path = 'input_images/'
    #image_name = 'vermeer'
    #image_file_format = '.jpg'
    image_name = image['image_name']
    image_file_format = image['image_file_format']
    compactness = image['compactness']
    n_segments = image['n_segments']
    thresh = image['thresh']
    t_lower = image['t_lower']
    t_upper = image['t_upper']
    cont_thickness = image['cont_thickness']

    file_name = image_name + image_file_format
    img = io.imread(image_path + file_name)
    #img = data.coffee()
    labels = segmentation.slic(img, compactness=compactness, n_segments=n_segments, start_label=1)

    # Create the RAG
    g = graph.rag_mean_color(img, labels)

    # Perform hierarchical merging
    labels2 = graph.merge_hierarchical(labels, g, thresh=thresh, rag_copy=False,
                                    in_place_merge=True,
                                    merge_func=merge_mean_color,
                                    weight_func=_weight_mean_color)

    # Map the final labels to colors
    out_original_colors = color.label2rgb(labels2, img, kind='avg', bg_label=0)
    io.imsave('output_images/' + image_name + ' -- color_merge.png', out_original_colors)

    output_image = np.zeros_like(img, dtype=np.uint8)
    for region in np.unique(labels2):
        mask = labels2 == region
        mean_color = img[mask].mean(axis=0)
        closest_color = closest_predefined_color(mean_color, predefined_colors)
        output_image[mask] = closest_color

    # Save and display the output image
    io.imsave('output_images/' + image_name + ' -- color_merge_forced_colors.png', output_image)
    f = open('output_images/' + image_name + '_parameters.txt', "w")
    f.write("compactness = " + str(compactness) + "\n" + "n_segments = " + str(n_segments) + "\n" + "thresh = " + str(thresh) + "\n" + "t_lower = " + str(t_lower) + "\n" + "t_upper = " + str(t_upper) + "\n" + "cont_thickness = " + str(cont_thickness) + "\n")
    f.close()

    #io.imshow(output_image)
    #io.show()


    get_contours(image_path + file_name)

    # BELOW IS THE OLD CONTOUR CODE
    # Convert to 0-1 range for use with mark_boundaries
    # contour_color_01 = convert_color_range(contour_color)

    # # Creating an image with only the contours against a black background
    # contours_only = np.zeros_like(img)
    # contours_only = segmentation.mark_boundaries(contours_only, labels2, color=contour_color_01, outline_color=contour_color_01, mode='thick')
    # contours_only = util.img_as_ubyte(contours_only)  # Convert the image to byte format

    # # Display the contours-only image
    # io.imshow(contours_only)
    # io.show()




    from PIL import Image

    # Load the images
    edge_image_path = 'output_images/' + image_name + ' -- contours.png'
    color_image_path = 'output_images/' + image_name + ' -- color_merge.png'
    forced_colors_image_path = 'output_images/' + image_name + ' -- color_merge_forced_colors.png'

    edge_image = Image.open(edge_image_path).convert("RGBA")
    color_image = Image.open(color_image_path).convert("RGBA")
    forced_colors_image = Image.open(forced_colors_image_path).convert("RGBA")

    # Convert the edge image to a numpy array
    edge_data = np.array(edge_image)



    # Make nearly black parts transparent
    edge_data = make_black_transparent(edge_data)

    # Convert back to an image
    edge_image_transparent = Image.fromarray(edge_data)

    # Combine the images
    combined_image = Image.alpha_composite(color_image, edge_image_transparent)

    # Save the combined image
    combined_image_path = 'output_images/' + image_name + ' -- combined all colors.png'
    combined_image.save(combined_image_path)

    forced_combined_image = Image.alpha_composite(forced_colors_image, edge_image_transparent)

    forced_colors_combined_image_path = 'output_images/' + image_name + ' -- combined forced colors.png'
    forced_combined_image.save(forced_colors_combined_image_path)

    # Display the combined image
    #combined_image.show()


Reading image: input_images/vermeer.jpg
Converting to grayscale
Done
Finding edges
Drawing contours with thickness 1
Saving image to: output_images/vermeer -- contours.png
Reading image: input_images/unknown 1640.jpg
Converting to grayscale
Done
Finding edges
Drawing contours with thickness 1
Saving image to: output_images/unknown 1640 -- contours.png
Reading image: input_images/turner.webp
Converting to grayscale
Done
Finding edges
Drawing contours with thickness 1
Saving image to: output_images/turner -- contours.png
Reading image: input_images/leo putz_Halbindianerin mit Früchten.jpg
Converting to grayscale
Done
Finding edges
Drawing contours with thickness 1
Saving image to: output_images/leo putz_Halbindianerin mit Früchten -- contours.png
Reading image: input_images/cavalcanti_mulata against a green background.jpg
Converting to grayscale
Done
Finding edges
Drawing contours with thickness 1
Saving image to: output_images/cavalcanti_mulata against a green background -- contours.png